In [ ]:
import sys
sys.path.append('../qick/qick_lib/')
sys.path.append('../soft/')

from mkids import *

import numpy as np

import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter
from numpy.fft import fft, fftshift

In [ ]:
# Initialize Firmware.
soc = MkidsSoc('./mkids_2x2_kidsim_v1.bit')

# Name the Chains.
soc['dual'][0]['name'] = '226 CH0 <-> 230 CH0'
soc['dual'][1]['name'] = '226 CH2 <-> 230 CH2'
soc['simu'][0]['name'] = '227 CH0 <-> 231 CH0'
soc['simu'][1]['name'] = '227 CH2 <-> 231 CH2'

# Print information.
print(soc)

In [ ]:
################################
### Processing Chain Example ###
################################
# Build processing chain.
chain = KidsChain(soc, dual=soc['dual'][0])

# Build simulation chain.
simu = SimuChain(soc, simu=soc['simu'][0])

In [ ]:
###############################
### Resonator Configuration ###
###############################
# Set quantization.
simu.analysis.qout(2)
simu.synthesis.qout(2)

# Disable all resonators.
simu.alloff()

# Channel separation.
fc = simu.analysis.fc_ch

# Set mixer.
simu.set_mixer_frequency(300)

# Set resonators.
fr0 = 300 + 10*fc
print("fr0 = {} MHz".format(fr0))

simu.enable(fr0+0.1,verbose=True)
#simu.enable(fr0+fc)
#simu.enable(fr0+2*fc)
#simu.enable(750)

#simu.disable(595)
#simu.disable(600)
#simu.disable(605)

In [ ]:
fmix = 600
chain.set_mixer_frequency(fmix)

# Data source.
chain.source("input")

# Set output tone.
ftone = 750
chain.set_tone(ftone, g=0.5)

# Decimation.
chain.set_decimation(1)
chain.analysis.qout(3)
chain.synthesis.qout(3)

# Transfer data.
[xi,xq] = chain.get_bin(ftone)
x = xi + 1j*xq
    
# Spectrum.
F = (np.arange(len(x))/len(x)-0.5)*chain.analysis.fs_ch    
w = np.hanning(len(x))
xw = x*w
YY = fftshift(fft(xw))
YYlog = 20*np.log10(abs(YY)/max(abs(YY)))
   
plt.plot(F,YYlog);

In [ ]:
plt.plot(xi)

In [ ]:
#######################
### Frequency Sweep ###
#######################
chain.analysis.qout(3)
chain.synthesis.qout(3)
fc = 324
df = 20
f,a,phi=chain.sweep(fc-df/2,fc+df/2,N=400,g=0.5)

plt.figure(dpi=150)
plt.plot(f,20*np.log10(a/max(a)))
#plt.ylim([-60,10])
plt.xlabel("Frequency [MHz]");
plt.ylabel("Amplitude [dB]");
#plt.savefig('vna_1.jpg')

plt.figure(dpi=150)
plt.plot(f,phi)
plt.xlabel("Frequency [MHz]");
plt.ylabel("Phase [rad]");

In [ ]:
plt.figure(dpi=150)
plt.plot(f,20*np.log10(a/max(a)))
plt.ylim([-60,10])
plt.xlabel("Frequency [MHz]");
plt.ylabel("Amplitude [dB]");

In [ ]:
############################
### Wide Frequency Sweep ###
############################
fstart = 0
fend = 1000

chain.analysis.qout(3)
chain.synthesis.qout(3)

# Number of pointes per sweep.
N = 100
#df = 1
#N = int(np.round((fend - fstart)/df))

# Use 80 % the available bandwidth per sweep.
fbw = 0.8*min(chain.analysis.fs,chain.synthesis.fs)

if (fend-fstart)>fbw:
    fstart = np.arange(fstart, fend, fbw)

# Total number  of points.
NT = len(fstart)*N

f_v = np.zeros(NT)
a_v = np.zeros(NT)
phi_v = np.zeros(NT)
for i,ff in enumerate(fstart):
    fend_ = ff+fbw
    if fend_ > fend:
        fend_ = fend
    print("i = {}, fstart = {} MHz, fend = {} MHz.".format(i, ff, fend_))
    
    # Sweep.
    f,a,phi=chain.sweep(ff,fend_,N=N,g=0.5)
    
    # Concat values.
    f_v[i*N:(i+1)*N] = f
    a_v[i*N:(i+1)*N] = a
    phi_v[i*N:(i+1)*N] = phi
    
plt.figure(dpi=150)
plt.plot(f_v,20*np.log10(a_v/max(a_v)))
plt.xlabel("Frequency [MHz]");
plt.ylabel("Amplitude [dB]");
#plt.savefig('wide-vna.jpg')

#plt.figure(dpi=150)
#plt.plot(f_v,phi_v)
#plt.xlabel("Frequency [MHz]");
#plt.ylabel("Phase [rad]");

In [ ]:
plt.figure(dpi=150)
plt.plot(f_v,20*np.log10(a_v/max(a_v)))
plt.xlabel("Frequency [MHz]");
plt.ylabel("Amplitude [dB]");
plt.xlim([200,300])
plt.ylim([-5,0])

In [ ]:
#########################
### Spectrum Analyzer ###
#########################

fmix = 1400
chain.set_mixer_frequency(fmix)

# Data source.
chain.source("input")

# Set output tone.
chain.synthesis.alloff()
chain.set_tone(1410.6, g=0.2)

# Decimation.
chain.set_decimation(1)
chain.analysis.qout(2)

# Frequency range.
fstart = 1409
fend   = 1412
f = np.arange(fstart, fend, chain.analysis.fc_ch)
print("Spectrum")
print("fstart = {} MHz, fend = {} MHz, fc = {} MHz".format(fstart, fend, chain.analysis.fc_ch))

# Set mixer to starting point.
#chain.analysis.set_mixer_frequency(-fstart)

# Frequency and amplitude vectors.
FF = []
AA = []
plt.figure(dpi=150);
for i,fck in enumerate(f):
    print("i = {}, fck = {} MHz".format(i,fck))
    
    # Transfer data.
    [xi,xq] = chain.get_bin(fck)
    x = xi + 1j*xq
    
    # Frequency vector.
    F = (np.arange(len(x))/len(x)-0.5)*chain.analysis.fs_ch    
    
    # Normalization factor.
    NF = (2**15)*len(F)

    w = np.hanning(len(x))
    xw = x*w
    YY = fftshift(fft(xw))
    YYlog = 20*np.log10(abs(YY)/NF)
    AA = np.concatenate((AA,YYlog))
    
    Fk = F+fck
    FF = np.concatenate((FF,Fk))
    plt.plot(Fk,YYlog);
plt.xlabel("Frequency [MHz]");
plt.ylabel("Amplitude [dB]");
#plt.savefig('wide-spectrum.jpg')
#plt.plot(FF,AA);

In [ ]:
###############################
### Zooming-in the spectrum ###
###############################
# Set decimation.
chain.set_decimation(1)

# Set quantization.
chain.analysis.qout(5)

# Data source.
chain.source("product")

# Get data.
fc = 110.59
[xi,xq] = chain.analysis.get_bin(fc, force_dds=True, verbose=True)
xi = xi[100:]
xq = xq[100:]
x = xi + 1j*xq

w = np.hanning(len(x))
xw = x*w
F = (np.arange(len(x))/len(x)-0.5)*chain.analysis.fs_ch*1000
Y = fftshift(fft(xw))

# Normalization factor.
NF = (2**15)*len(F)

plt.figure(1,dpi=150)
plt.plot(F,20*np.log10(abs(Y)/NF))
plt.xlabel("F [kHz]");
plt.ylabel("Amplitude [dB]");
#plt.savefig('zoom-spectrum-2.jpg')

In [ ]:
chain.analysis.get_mixer_frequency()

In [ ]:
########################
### Delay estimation ###
########################
#chain.set_mixer_frequency(290)

# Delta frequency per sweep.
df = 5

# Number of points per sweep.
N = 500

# Starting frequency points.
fstart = 324

# Frequency Sweep.
f,a,phi = chain.sweep(fstart-df/2,fstart+df/2,N=N, g=0.5, set_mixer=False)
df, dt = chain.phase_slope(f, phi)
 
print(" ")
print("df = {} MHz, dt = {} us".format(df, dt))

plt.figure(dpi=180)
plt.plot(f,phi)
plt.xlabel("Frequency [MHz]");
plt.ylabel("Phase [rad]");
plt.gca().xaxis.set_major_formatter(StrMethodFormatter('{x:,.2f}'))
plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))
#plt.savefig('phase-short.jpg')

In [ ]:
##############################
### Phase Correction by DT ###
##############################
# DT is in us.
DT = -22-0.635

phi_u, phi_dt = chain.phase_correction(f, phi, DT=DT, phase_cal=j_avg)

plt.figure(dpi=150)
plt.plot(f,phi)
plt.xlabel("Frequency [MHz]");
plt.ylabel("Phase [rad]");
plt.title("Original Phase");

plt.figure(dpi=150)
plt.plot(f,phi_u)
plt.xlabel("Frequency [MHz]");
plt.ylabel("Phase [rad]");
plt.title("Unwrap Phase");

plt.figure(dpi=150)
plt.plot(f,phi_dt)
plt.xlabel("Frequency [MHz]");
plt.ylabel("Phase [rad]");
plt.title("Correctred phase by DT = {:.6f} us".format(DT));

In [ ]:
#################################
### Overall delay computation ###
#################################
data = chain.phase_fit(f, phi_dt, jumps=False)

m = data['fits'][0]['slope']
x = data['fits'][0]['data']['x']
y = data['fits'][0]['data']['y']
fn = data['fits'][0]['data']['fn']
    
print("Slope = = {} us".format(m/(2*np.pi)))
plt.figure(dpi=150);
plt.plot(x, y, '.', x, fn, '--k');
plt.xlabel("Frequency [MHz]");
plt.ylabel("Phase [rad]");
plt.title("Slope = {:.5} ns".format(1000*m/(2*np.pi)));
#plt.savefig('phase-slope-2.jpg')

In [ ]:
####################################
### Jump-based delay computation ###
####################################
data = chain.phase_fit(f, phi_dt,gap=5)

# Sampling period (ns).
ts = 1000/chain.analysis.fs

m_avg = 0
for i in range(len(data['fits'])):
    m  = data['fits'][i]['slope']
    x  = data['fits'][i]['data']['x']
    y  = data['fits'][i]['data']['y']
    fn = data['fits'][i]['data']['fn']
    
    m_avg = m_avg + m
      
    print("Slope [{}]\t= {:.5f} ns".format(i,1000*m/(2*np.pi)))
    #plt.figure(dpi=180);
    #plt.plot(x, y, '.', x, fn, '--k');
    #plt.xlabel('Frequency [MHz]')
    #plt.ylabel('Phase [rad]')
    #plt.title('Slope [{}] = {:.5} ns'.format(i,1000*m/(2*np.pi)))
    #plt.savefig('phase-slope-jump-{}.jpg'.format(i))
    
m_avg = m_avg/len(data['fits'])
print("Average Slope\t= {:.5f} ns".format(1000*m_avg/(2*np.pi)))
print(" ")
    
for i in range(len(data['jump']['value'])):
    jv = data['jump']['value'][i]
    
    print("Jump [{}]\t= {:.5f} rad, {:.5f} ns, {:5f} samples".format(i, jv, 1000*jv/(2*np.pi), 1000*jv/(2*np.pi*ts)))
    
j_avg = np.mean(data['jump']['value'])
print("Average Jump\t= {:.5f} rad, {:.5f} ns, {:.5f} samples".format(j_avg, 1000*j_avg/(2*np.pi), 1000*j_avg/(2*np.pi*ts)))

In [ ]:
1/23/2